# Classificazione immagini

In [6]:
import numpy as np
import albumentations as A
from PIL import Image
import os

import matplotlib.pyplot as plt

import torch
import keras

## Dataset

prima di tutto definiamo in automatico le classi tramite il nome delle cartelle

In [12]:
cartella_data = "data"
batch_size = 16
augmentation = True

# salvo il nome delle classi in una lista
class_names = os.listdir(cartella_data)
class_names.sort() # ordino la lista in ordine alfabetico

print("le classi sono: ", class_names)
print("numero di classi: ", len(class_names))

le classi sono:  ['biscotti', 'caffè', 'modella', 'pizza']
numero di classi:  4


ora visualizziamo lo sbilanciamento delle classi

In [13]:
# creo un dizionario con le classi e il numero di immagini per classe
classi = {}
for classe in class_names:
    classi[classe] = len(os.listdir(os.path.join(cartella_data, classe)))

# stampo il numero di immagini per classe, e la relativa percentuale
for classe in class_names:
    print("classe: ", classe, "\nnumero di immagini: ", classi[classe], "\npercentuale: ", round(classi[classe]/sum(classi.values()), 3), "\n______________________")

classe:  biscotti 
numero di immagini:  10 
percentuale:  0.159 
______________________
classe:  caffè 
numero di immagini:  25 
percentuale:  0.397 
______________________
classe:  modella 
numero di immagini:  15 
percentuale:  0.238 
______________________
classe:  pizza 
numero di immagini:  13 
percentuale:  0.206 
______________________


costruiamo un generatore di batch

In [ ]:
def generatore_batch_classification(cartella = "data", batch_size = 16, augmentation = True):
    '''
    Generatore che restituisce un batch di immagini e le rispettive label

    Parametri
    ----------
    cartella_data : string
        path della cartella contenente le immagini
    batch_size : int
        dimensione del batch di immagini che vogliamo ottenere
    augmentation : bool
        se True, applica data augmentation alle immagini
    
    Yields
    ------
    camera_batch : numpy array
        batch di immagini della camera
    classi_batch : numpy array
        batch di classi corrispondenti alle immagini
    '''
    buffer = batch_size
    dimensione_output = (256,256)

    for i in range(0, len(class_names), buffer):
        
        #gestisco il caso in cui l'ultimo batch non sia completo
        if i + buffer > len(class_names):
            i = len(class_names) - buffer

        camera_batch = []
        classi_batch = []
        for j in range(i, i + buffer):
            with Image.open(os.path.join(cartella_data, class_names[j])) as camera:
                camera = np.array(camera)
                # Data augmentation, agendo solo sul colore, contrasto, luminosità saturazione
                if augmentation:
                    transform = A.Compose([
                        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.2, p=0.5),
                        A.RandomGamma(gamma_limit=(80, 120), p=0.5),
                        A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.5),
                        A.HorizontalFlip(p=0.5),
                        A.VerticalFlip(p=0.5),
                        A.RandomRotate90(p=0.5),
                        A.Transpose(p=0.5),
                        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.5),
                        A.RandomCrop(height=256, width=256, p=0.5),
                        A.Resize(height=256, width=256, p=1),
                        A.Normalize(p=1.0),
                        ])
                    camera = transform(image=camera)['image']
                else:
                    transform = A.Compose([
                        A.Resize(height=256, width=256, p=1),
                        A.Normalize(p=1.0),
                        ])
                    camera = transform(image=camera)['image']

            camera_batch.append(camera)
            classi_batch.append(class_names[j])
        yield np.array(camera_batch), np.array(classi_batch)

# test
for camera_batch, classi_batch in generatore_batch_classification(cartella = "data", batch_size = 16, augmentation = True):
    print("dimensione del batch: ", camera_batch.shape)
    print("dimensione del batch: ", classi_batch.shape)
    break

In [11]:
# test del generatore

for camera_batch, classi_batch in generatore_batch_classification(cartella_data, batch_size, augmentation):
    print(camera_batch.shape)
    print(classi_batch.shape)
    break


IndexError: list index out of range